# Applied Data Science Capstone

## Week 3: Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [1]:
# following this youtube tutorial: https://www.youtube.com/watch?v=ng2o98k983k
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [3]:
table = soup.find('table')
#print(table.prettify())

In [4]:
output_rows = []
for table_row in table.findAll('tr'):

    output_row = []
    #headings will be defined in the data frame with "column = [...]"
    #columns = table_row.findAll('th')
    #for column in columns:
    #    output_row.append(column.text.replace('\n', ' ').strip())
    
    columns = table_row.findAll('td')
    for column in columns:
        output_row.append(column.text.replace('\n', ' ').strip())
    output_rows.append(output_row)

output = pd.DataFrame(output_rows,columns = ['PostalCode','Borough','Neighborhood'])

# delete all rows where the borough is not assigned
output = output[output.Borough != 'Not assigned'] 

# drop the headings - they are showing as null
output.dropna(subset=["PostalCode"], axis=0, inplace=True) 

# replace Not assigned neighborhoods by their borough name 
#it affects Queen's Park for example
output["Neighborhood"].replace("Not assigned", output["Borough"], inplace=True)

#combining neighborhoods
#it affects M5A for example, more than 1 neighborhood per postalcode
output = output.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()

#first 20 records
output.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [5]:
output.shape

(103, 3)